In [1]:
from sage.graphs.connectivity import connected_components
from sage.graphs.connectivity import connected_components_number

#m x n mreža, a št. izbrisanih vozlov, b št.izrbirsanih povezav
def mreza(m,n,a,b):
    mreza = graphs.Grid2dGraph(m,n)
    if a > mreza.order():
        print("Za ukaz je na voljo premalo vozlov.")
    else:
        i = 0
        while i < a:
            mreza.delete_vertex(mreza.random_vertex())
            i = i+1
        i = 0
    if b > mreza.size():
        print("Za ukaz je na voljo premalo povezav.")
    else:
        while i < b:
            mreza.delete_edge(mreza.random_edge())
            i = i+1
    return mreza

#reši CLP problem in vrne k
def najkrajsa_razdalja(G, st_centrov):
    K = st_centrov
    razdalje = G.distance_all_pairs()

    p = MixedIntegerLinearProgram(maximization=False)
    x = p.new_variable(binary=True) #x_uv = 1 če mesto u spada k skladišču v (mestu v s skladiščem)
    y = p.new_variable(binary=True) # y_v = 1 če je v mestu v skladišče

    p.set_objective(p['R']) # največja razdalja je spremenljivka

    for u in G:
        p.add_constraint(sum(x[u, v] for v in G) == 1) #za vsako mesto u bo veljalo, da spada pod neko območje mesta v s skladiščem

    p.add_constraint(sum(y[v] for v in G) == K) #vsota skladišč je enaka K

    for u in G:
        for v in G:
            p.add_constraint(x[u, v] <= y[v]) #ne sme se zgoditi, da mesto u pade v območje mesta v, v mestu v pa sploh ni skladišča

    for u in G:
        for v in G:
            if v in razdalje[u]:
                p.add_constraint(razdalje[u][v] * x[u, v] <= p['R']) # če sta vozlišči v isti povezani komponenti, potem omejimo največjo razdaljo                                                                        do skladišča
            else:
                p.add_constraint(x[u, v] == 0) # sicer mesto u ne more pripadati skladišču v
    max_razdalja = p.solve()
    skladisca = [k for k, v in p.get_values(y).items() if v == 1]
    #print(skladisca)
    return max_razdalja


#G = mreza(5,3,3,2)
#slika=G.show()
#najkrajsa_razdalja(G,5)

#print(connected_components(G)) #večdelni graf zapiše po ločenih delih
#print(connected_components_number(G)) #št. delov

In [2]:
#kako se optimalna vrednost spreminja glede na k
def opt_vrednost_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_vrednosti = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1): ## težave pri določanju najmanjšega i???
        razdalja = round(najkrajsa_razdalja(G, i)) #round za zaokrževanje števil
        seznam_vrednosti.append((razdalja))
    return seznam_vrednosti

#vrne optimalno vrednost R za več ponovitev
#FIKSNO: velikost mreže, število izbrisanih povezav in vozlišč
#SPREMINJAVA: k
def opt_vrednost_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        G = mreza(m,n,a,b)
        razdalje = opt_vrednost_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(razdalje)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return povprecja

opt_vrednost_za_vec_ponovitev(3,4,2,2,2,10)

[(0, 2), (1, 1)]
[(1, 1), (2, 0)]
[(1, 2)]
[(1, 1), (1, 2)]
[(0, 1), (2, 3)]
[(0, 3), (1, 1)]
[(0, 3), (2, 1)]
[(0, 1)]
[(1, 1), (1, 3)]
[(0, 2), (1, 2)]
[(0, 0), (2, 2)]


[3.5, 2.7777777777777777]

In [3]:
import time

#čas izvajanja v odvisnosti od k
def cas_izvajanja_k(m,n,a,b,k):
    G = mreza(m,n,a,b)
    seznam_casov = []
    stevilo_komponent = connected_components_number(G)
    for i in range(stevilo_komponent,k+1):
        zacetni = time.time()
        najkrajsa_razdalja(G,i)
        koncni = time.time() - zacetni
        seznam_casov.append((koncni))
    return seznam_casov

cas_izvajanja_k(3,3,1,1,5)

#vrne povprečne čase(od n ponovitev) za različne k
#FIKSNO: velikost mreže, št. izbrisanih povezav in vozlišč
#SPREMINJAMO: k
def cas_izvajanja_za_vec_ponovitev(m,n,a,b,max_stevilo_centrov,stevilo_ponovitev):
    seznam = []
    for i in range(0, stevilo_ponovitev):
        G = mreza(m,n,a,b)
        casi = cas_izvajanja_k(m,n,a,b,max_stevilo_centrov)
        seznam.append(casi)

    for i in range(len(seznam)):
        while len(seznam[i]) < max_stevilo_centrov:
            seznam[i] = [None] + seznam[i]

    povprecja = []
    for j in range(max_stevilo_centrov):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)

    return povprecja

cas_izvajanja_za_vec_ponovitev(3,4,2,2,3,10)

[(1, 2)]
[(0, 1), (2, 1)]
[(0, 0), (1, 2), (2, 0)]
[(0, 0), (0, 1), (1, 2), (2, 0)]
[(0, 0), (0, 1), (0, 2), (1, 0), (2, 1)]
[(1, 1), (2, 2)]
[(0, 2), (1, 1), (2, 2)]
[(1, 2)]


[(0, 1), (1, 2)]
[(0, 1), (1, 3), (2, 2)]
[(2, 0)]


[(0, 0), (1, 2)]
[(0, 1), (1, 3), (2, 1)]
[(1, 2), (2, 3)]


[(0, 1), (1, 1), (2, 3)]
[(1, 1), (2, 3)]
[(0, 1), (1, 1), (2, 3)]
[(0, 2), (2, 0), (2, 2)]
[(0, 2), (1, 1)]
[(0, 0), (0, 3), (1, 1)]
[(0, 1), (1, 2)]
[(0, 0), (1, 2), (2, 1)]
[(0, 0), (1, 2)]
[(0, 2), (1, 0), (1, 2)]
[(0, 3), (1, 1)]


[(0, 0), (0, 3), (1, 0)]


[0.04117906093597412, 0.04338179694281684, 0.042748093605041504]

In [4]:
#kako se oprimalna vrednost spreminja glede na št. izbrisanih vozlov a
def opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_vrednosti = []
    for i in range(0,max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            razdalja = round(najkrajsa_razdalja(G, k)) #round za zaokrževanje števil 
            seznam_vrednosti.append((razdalja))
        else:
            seznam_vrednosti.append(None)
            
    return seznam_vrednosti

opt_vrednost_vozli(5,5,15,2,3)

# !NEKI PAMETNGA BO TREBA NAREST Z None-i, ker primerjamo jabolka pa hruske pri razdrobljenih grafih pri relativno majhnih k!
#fukncija vrne opt. razdalje R glede na število odstranjenih vozlišč
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlišč
def R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        razdalje = opt_vrednost_vozli(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(razdalje)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return povprecja

R_v_odvisnosti_od_spreminjanje_stevila_vozlisc(5,5,15,2,3,3)

[(1, 2), (3, 1), (4, 1)]


[(0, 2), (1, 2), (4, 2)]


[(1, 1), (1, 3), (3, 2)]


[(1, 1), (1, 4), (3, 1)]


[(0, 2), (3, 1), (3, 4)]


...

[(1, 0), (1, 4), (4, 2)]


[(0, 4), (2, 1), (4, 2)]


[(1, 1), (2, 3), (3, 0)]
[(0, 0), (1, 2), (4, 4)]


[(1, 2), (2, 0), (3, 3)]
[(1, 1), (2, 3), (3, 0)]


[3.0,
 2.6666666666666665,
 2.3333333333333335,
 2.6666666666666665,
 3.0,
 3.6666666666666665,
 2.6666666666666665,
 3.5,
 3.0,
 2.5,
 None,
 None,
 None,
 3.0,
 None,
 None]

In [5]:
import time

#funkcija drugacna od cas_izvajanja_k(m,n,a,b,k)
#čas izvajanja v odvisnosti od k
def cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k):
    seznam_casov = []
    for i in range(max_st_izbrisanih_vozlisc + 1):
        G = mreza(m,n,i,b)
        stevilo_komponent = connected_components_number(G)
        if stevilo_komponent <= k:
            zacetni = time.time()
            najkrajsa_razdalja(G,k)
            koncni = time.time() - zacetni
            seznam_casov.append(koncni)
        else:
            seznam_casov.append(None)
    return seznam_casov

cas_izvajanja_k(3,3,1,1,5)

#funkcija vrne povprečne čase (od n ponovitev) glede na spreminjanje vozlov
#FIKSNO:velikost mreze, K, st. izbrisanih povezav
#SPREMINJAM: st_izbrisanih vozlov
def cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(m,n,max_st_izbrisanih_vozlisc,b,k,stevilo_ponovitev):
    seznam = []
    for i in range(stevilo_ponovitev):
        casi = cas_izvajanja_k2(m,n,max_st_izbrisanih_vozlisc,b,k)
        seznam.append(casi)

    povprecja = []
    for j in range(max_st_izbrisanih_vozlisc + 1):
        vsota = 0
        stevec = 0
        for i in range(stevilo_ponovitev):
            if seznam[i][j] != None:
                vsota += seznam[i][j]
                stevec += 1
        if stevec == 0:
            povprecja.append(None)
        else:
            povprecja.append(vsota/stevec)
    return povprecja

cas_v_odvisnosti_od_spreminjanja_stevila_vozlisc(5,5,15,2,3,3)

[(1, 1)]
[(1, 0), (1, 1)]
[(0, 0), (1, 0), (2, 2)]
[(0, 0), (0, 1), (1, 0), (2, 2)]
[(0, 0), (0, 1), (1, 0), (1, 1), (2, 1)]


[(0, 2), (3, 0), (3, 3)]


[(0, 3), (2, 0), (3, 2)]


[(1, 1), (2, 4), (4, 1)]


[(0, 1), (3, 1), (3, 4)]


...

[(0, 2), (2, 0), (3, 3)]


[(0, 0), (1, 4), (4, 2)]
[(0, 0), (0, 2), (4, 2)]


[(1, 2), (1, 4), (3, 2)]
[(1, 3), (2, 0), (3, 2)]


[(0, 0), (2, 3), (3, 2)]
[(0, 3), (2, 0), (4, 3)]


[4.747821728388469,
 4.130467255910237,
 3.0132040977478027,
 2.196422894795736,
 1.622791051864624,
 1.096357027689616,
 0.32151540120442706,
 0.09397586186726888,
 0.11115682125091553,
 0.03382134437561035,
 0.045514822006225586,
 None,
 None,
 0.04423713684082031,
 None,
 None]